In [1]:
import os
import pandas as pd

In [247]:
def get_df_class_id_model(df):
    def split_path(row):
        path = row.name[0].strip()
        row['id'] = path
        row['class'] = path
        if(len(path) > 3):
            row['id'] = path.split(' ')[-1] 
            row['class'] = path.split(' ')[-2]
        return row
    df = df.apply(split_path, axis=1)
    df = df.reset_index()
    df = df.drop(columns='path')
    df = df.set_index(['class', 'id', 'model', 'run id'])
    return df


def get_df_aggregate(df, index, df_slice=None):
    if df_slice:
        df = df.loc[df_slice]
    all_index = ['class', 'id', 'run id']
    values = 'roc value'
    columns = 'model'
    aggfunc = ['mean', 'std']
    df = df.reset_index()
    df = df.pivot_table(index=index, values=values, aggfunc=aggfunc, columns=columns)
    for i in all_index:
        if(not i in index):
            df[i] = 'all'
    df = df.reset_index()
    df = df.set_index(['class', 'id', 'run id'])
    return df

def get_df_from_path(path):
    df = pd.read_hdf(path)
    df = get_df_class_id_model(df)
    return df



## Individual Analysis

In [255]:
path_input_individual = "/data/output/rocs/results_01_individual.h5"
df_individual = get_df_from_path(path_input_individual)
df_individual

fit time    roc time  roc value
class id   model         run id                                    
valve id00 Baseline      0        922.820497  305.504915   0.281336
                         1       1028.808547  304.421828   0.338324
                         2       1085.586410  185.778189   0.359862
           MFCCMix M     0        134.024107   34.730547   0.524822
                         1        137.579356   33.928242   0.495728
...                                      ...         ...        ...
pump  id04 MFCCMix S+C   1         70.286250   24.262587   0.633200
                         2         69.792470   24.155285   0.630900
           MFCCMix M+S+C 0         69.645075   24.023390   0.883500
                         1         70.491663   23.508649   0.859800
                         2         95.368363   31.551257   0.917800

[384 rows x 3 columns]

In [ ]:
df_slice = (slice(None), slice(None), ['Baseline', 'MFCCMix M+S+C'])

In [277]:
index = []
df_individual_agg = get_df_aggregate(df_individual, index, df_slice)
df_individual_agg

index      mean                     std              
model                        Baseline MFCCMix M+S+C  Baseline MFCCMix M+S+C
class id  run id                                                           
all   all all     roc value  0.725884      0.831156  0.184295      0.108667

In [278]:
index = ['class']
df_individual_agg = get_df_aggregate(df_individual, index, df_slice)
df_individual_agg


mean                     std              
model              Baseline MFCCMix M+S+C  Baseline MFCCMix M+S+C
class  id  run id                                                
fan    all all     0.807394      0.855882  0.142453      0.077957
pump   all all     0.813318      0.890826  0.102721      0.040977
slider all all     0.797661      0.857467  0.138882      0.130620
valve  all all     0.485163      0.720448  0.102791      0.083423

In [279]:
index = ['class', 'id']
df_individual_agg = get_df_aggregate(df_individual, index, df_slice)
df_individual_agg

mean                     std              
model               Baseline MFCCMix M+S+C  Baseline MFCCMix M+S+C
class  id   run id                                                
fan    id00 all     0.616283      0.729533  0.010579      0.007349
       id02 all     0.867555      0.905347  0.010330      0.030122
       id04 all     0.760110      0.887637  0.013081      0.009142
       id06 all     0.985630      0.901011  0.001251      0.014020
pump   id00 all     0.771529      0.948066  0.005527      0.005446
       id02 all     0.711279      0.847036  0.014458      0.015503
       id04 all     0.971767      0.887033  0.008346      0.029161
       id06 all     0.798699      0.881167  0.038834      0.012739
slider id00 all     0.983328      0.993356  0.003038      0.004962
       id02 all     0.810209      0.923078  0.017032      0.005850
       id04 all     0.784939      0.853501  0.019197      0.027009
       id06 all     0.612170      0.659934  0.040258      0.018479
valve  id00 all     0.326507      0.792764  0.040574      0.041856
       id02 all     0.564190      0.657500  0.028965      0.015290
       id04 all     0.551181      0.800093  0.013876      0.007666
       id06 all     0.498773      0.631435  0.038656      0.032316

In [280]:
index = ['class', 'id', 'run id']
df_individual_agg = get_df_aggregate(df_individual, index, df_slice)
df_individual_agg

mean              
model               Baseline MFCCMix M+S+C
class  id   run id                        
fan    id00 0       0.625063      0.731402
            1       0.604537      0.721429
            2       0.619249      0.735767
       id02 0       0.868188      0.930129
            1       0.856922      0.914091
            2       0.877554      0.871820
       id04 0       0.768257      0.878741
            1       0.767051      0.887163
            2       0.745021      0.897006
       id06 0       0.986472      0.884953
            1       0.986226      0.910820
            2       0.984193      0.907260
pump   id00 0       0.766052      0.946648
            1       0.771431      0.943469
            2       0.777104      0.954081
       id02 0       0.701323      0.864784
            1       0.727863      0.836133
            2       0.704651      0.840192
       id04 0       0.981400      0.883500
            1       0.967200      0.859800
            2       0.966700      0.917800
       id06 0       0.762303      0.868608
            1       0.794214      0.880815
            2       0.839581      0.894079
slider id00 0       0.980708      0.991313
            1       0.982617      0.989742
            2       0.986657      0.999014
       id02 0       0.820940      0.921250
            1       0.790571      0.929624
            2       0.819117      0.918360
       id04 0       0.778500      0.857688
            1       0.806527      0.878172
            2       0.769789      0.824643
       id06 0       0.590330      0.680722
            1       0.658629      0.645373
            2       0.587552      0.653705
valve  id00 0       0.281336      0.840477
            1       0.338324      0.762234
            2       0.359862      0.775581
       id02 0       0.530833      0.656806
            1       0.578750      0.642569
            2       0.582986      0.673125
       id04 0       0.550278      0.808819
            1       0.565486      0.794444
            2       0.537778      0.797014
       id06 0       0.466458      0.603750
            1       0.488264      0.623611
            2       0.541597      0.666944

## Combined Analysis

In [266]:
path_input_combined = "/data/output/rocs/results_03_combined.h5"
df_combined = get_df_from_path(path_input_combined)
df_combined

roc value
class id   model       run id           
fan   id00 Baseline    0        0.589753
                       1        0.584290
                       2        0.579448
           MFCCMix C   0        0.572602
                       1        0.582110
...                                  ...
valve id06 MFCCMix S   1        0.300556
                       2        0.370000
           MFCCMix S+C 0        0.222361
                       1        0.320625
                       2        0.245069

[384 rows x 1 columns]

In [273]:
index = []
df_combined_agg = get_df_aggregate(df_combined, index, df_slice)
df_combined_agg

index      mean                     std              
model                        Baseline MFCCMix M+S+C  Baseline MFCCMix M+S+C
class id  run id                                                           
all   all all     roc value  0.576767      0.598701  0.186462      0.188351

In [274]:
index = ['class']
df_combined_agg = get_df_aggregate(df_combined, index, df_slice)
df_combined_agg

mean                     std              
model              Baseline MFCCMix M+S+C  Baseline MFCCMix M+S+C
class  id  run id                                                
fan    all all     0.480623      0.661890  0.134830      0.068500
pump   all all     0.597953      0.596907  0.236090      0.284004
slider all all     0.728740      0.678314  0.163765      0.157607
valve  all all     0.499751      0.457692  0.074684      0.091784

In [275]:
index = ['class', 'id']
df_combined_agg = get_df_aggregate(df_combined, index, df_slice)
df_combined_agg

mean                     std              
model               Baseline MFCCMix M+S+C  Baseline MFCCMix M+S+C
class  id   run id                                                
fan    id00 all     0.584497      0.577375  0.005156      0.013096
       id02 all     0.423729      0.720116  0.134793      0.075857
       id04 all     0.596963      0.649546  0.017633      0.018707
       id06 all     0.317301      0.700522  0.011155      0.035568
pump   id00 all     0.752441      0.831630  0.004861      0.011022
       id02 all     0.782485      0.776587  0.011776      0.032538
       id04 all     0.227800      0.144400  0.098549      0.018616
       id06 all     0.629085      0.635012  0.053254      0.041240
slider id00 all     0.978015      0.822105  0.006345      0.142722
       id02 all     0.733951      0.792521  0.006736      0.031734
       id04 all     0.637619      0.618220  0.010218      0.019695
       id06 all     0.565375      0.480411  0.039742      0.010539
valve  id00 all     0.475626      0.477156  0.128343      0.042055
       id02 all     0.563310      0.564815  0.030832      0.012610
       id04 all     0.498426      0.444398  0.013264      0.035511
       id06 all     0.461644      0.344398  0.062971      0.076459

In [268]:
index = ['class', 'id', 'run id']
df_combined_agg = get_df_aggregate(df_combined, index, df_slice)
df_combined_agg

mean              
model               Baseline MFCCMix M+S+C
class  id   run id                        
fan    id00 0       0.589753      0.589041
            1       0.584290      0.579877
            2       0.579448      0.563209
       id02 0       0.578068      0.645836
            1       0.329133      0.797457
            2       0.363987      0.717057
       id04 0       0.589130      0.629657
            1       0.584605      0.666791
            2       0.617156      0.652192
       id06 0       0.319595      0.709893
            1       0.305177      0.661206
            2       0.327131      0.730466
pump   id00 0       0.756663      0.827082
            1       0.753533      0.844198
            2       0.747127      0.823610
       id02 0       0.769337      0.784677
            1       0.786056      0.804318
            2       0.792062      0.740768
       id04 0       0.328800      0.123600
            1       0.222700      0.150100
            2       0.131900      0.159500
       id06 0       0.603422      0.680411
            1       0.593522      0.624760
            2       0.690311      0.599865
slider id00 0       0.979177      0.881991
            1       0.971168      0.925128
            2       0.983698      0.659197
       id02 0       0.736509      0.811303
            1       0.726311      0.810377
            2       0.739034      0.755881
       id04 0       0.646983      0.637072
            1       0.626720      0.619808
            2       0.639155      0.597778
       id06 0       0.538568      0.477465
            1       0.546522      0.471658
            2       0.611034      0.492110
valve  id00 0       0.327449      0.449262
            1       0.551868      0.525528
            2       0.547560      0.456677
       id02 0       0.589444      0.550347
            1       0.529306      0.570625
            2       0.571181      0.573472
       id04 0       0.511667      0.403819
            1       0.498472      0.459583
            2       0.485139      0.469792
       id06 0       0.394028      0.388542
            1       0.518611      0.256111
            2       0.472292      0.388542